In [1]:
import numpy as np
import pandas as pd
import random
import math
import collections
from sklearn import preprocessing

In [2]:
#Read txt file
data= pd.read_csv("/Users/SKU/Desktop/fall2017/data mining/Project 3/project3_dataset2.txt", sep="\t", header=None)
# print(data.head())

In [3]:
# Separate Attribute and Label
labels =data[data.columns[-1]]
data = data.ix[:, :len(data.columns)-2]


/Users/SKU/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# Handeling Categorical Data

#Select Categorical Columns from the dataframe
obj_df = data.select_dtypes(include=['object']).copy()

#One-Hot Encoding for categorical data
for idx in obj_df.columns:
    obj_df= pd.get_dummies(obj_df,columns=[idx],drop_first=False)

obj_dataset = obj_df.values.tolist()

In [5]:
# Handeling Numerical Data
#Select numerical data from the data
num_df = data.select_dtypes(include=['int64','float64']).copy()

# Normalize Numeric Data
if (len(num_df.columns) !=0):
    num_dataset = preprocessing.normalize(num_df, norm='l2')
    num_dataset
else:
    num_dataset = []
print(len(num_dataset))

462


In [6]:
# Dataset Format to be used for algorithm
if len(num_dataset)!= 0:
    dataset = np.column_stack((num_dataset, obj_dataset))
    dataset = np.column_stack((dataset, labels))
else:
    dataset = np.column_stack((obj_dataset, labels))
(len(dataset[0]))

11

In [7]:
# calculate number features and size of training set
total_features = len(dataset[0])
features = math.ceil(0.2 * total_features)
#features = math.ceil(math.sqrt(total_features))

# get all labels from the dataset
labels = set(col[-1] for col in dataset)
labels = list(labels)
print(labels)

def select_traindataset(new_dataset, set_count):
    train_data = []
    rows = random.sample(range(0, len(new_dataset)-1), set_count)
    for row in rows:
        train_list = new_dataset[row]
        list2=[]
        for c in cols:
            list2.append(train_list[c])
        train_data.append(list2)         
    return train_data 

#divide entire dataset into training and testing
def divide_dataset():
    working_set = list(dataset)
    row_count = math.ceil(len(working_set) * 0.3)
    test_dataset = []
    for row in range(0, row_count):
        r = random.randint(0, len(working_set)-1)
        test_dataset.append(working_set[r])
        working_set.pop(r)
    train_dataset = working_set
    return train_dataset, test_dataset

[0.0, 1.0]


In [8]:
# divide entire training data into n folds
def cross_validation(n_folds):
    updated_datasets = list(dataset)

    size_fold = math.floor(len(updated_datasets)/n_folds)
    new_dataset = []

    for i in range(0, n_folds):
        count = 0
        fold = []
        while count < size_fold:
            r = random.randint(0,len(updated_datasets)-1)
            fold.append(updated_datasets[r])
            updated_datasets.pop(r)
            count +=1
        new_dataset.append(fold)
        
    return new_dataset

In [9]:
# to select 20% of the total feature for each node
def select_features(index_list):
    cols = random.sample(range(0, len(train_set[0])-2), features)
    cols.append(len(train_set[0])-1)
    fdata = []
    for row in index_list:
        list1 = train_set[row]
        list2 = []
        for c in cols:
            list2.append(list1[c]) 
        fdata.append(list2)         
    return fdata 
                

In [10]:
# Split a dataset based on given attribute value
def split(idx, val, dataset1, index_list):
    lft, rt = list(), list()
    idx_lft, idx_rt = list(), list()
    row_idx = 0
    it = iter(index_list)
    for row in dataset1:
        if row[idx] < val:
            lft.append(row)
            idx_lft.append(next(it))
        else:
            rt.append(row)
            idx_rt.append(next(it))
        row_idx +=1
    return lft, rt , idx_lft, idx_rt  

# Calculate the Gini index for splitted dataset
def gini_idx(attr_grps):
    
    # count all possible samples at split point
    subtrees = []
    subtrees.append(attr_grps[0])
    subtrees.append(attr_grps[1])
    total_instances = float(sum([len(sub) for sub in subtrees]))
    
    #sum weighted Gini index for each group
    gini = 0.0
    for sub in subtrees:
        size = float(len(sub))
        
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        
        # score the group based on the score for each class
        for class_val in labels:
            p = [row[-1] for row in sub].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / total_instances)

    return gini


# Select the best split point for a dataset with different attributes
def get_split(node_set, attr_list, index_list):
    b_idx, b_val, b_gini_score, b_attr_grp = 9999, 9999, 9999, None
    #update features selecting randomly
    feature_set = select_features(index_list)
    for idx in range(len(feature_set[0])-1):
        countloop = 0
        for row in feature_set:
            countloop +=1
            attr_grps = split(idx, row[idx], feature_set, index_list) 
            gini = gini_idx(attr_grps)
            if gini < b_gini_score:
                b_idx, b_val, b_gini_score, b_attr_grp = idx, row[idx], gini, attr_grps
    return {'idx':b_idx, 'a_val':b_val, 'grps':b_attr_grp, 'gini':  b_gini_score }

In [11]:
# Create a terminal node value
def terminal(attr_grp):
    results = [row[-1] for row in attr_grp]
    return max(set(results), key=results.count)

# Create child splits or terminal nodes
def node_split(node, depth, attr_list, max_depth):
    lft, rt, lft_idx, rt_idx = node['grps']
    del(node['grps'])
    
    # check for a no split
    if not lft or not rt:
        node['lft'] = node['rt'] = terminal(lft + rt)
        return
    
     # check for max depth
    if depth >= max_depth:
        node['lft'], node['rt'] = terminal(lft), terminal(rt)
        return
    
    # process only left child
    if len(attr_list) == len(lft[0])-1:
        node['lft'] = terminal(lft)
        
    else:
        node['lft'] = get_split(lft, attr_list , lft_idx)
        attr_list.append(node['lft']['idx'])
        node_split(node['lft'], depth+1,attr_list,max_depth)
        
    # process only right child
    if len(attr_list) == len(rt[0])-1:
        node['rt'] = terminal(rt)
    else:
        node['rt'] = get_split(rt, attr_list , rt_idx)
        attr_list.append(node['rt']['idx'])
        node_split(node['rt'], depth+1, attr_list, max_depth)
        
# Build a decision tree
def build_tree(train_dataset, attr_list, index_list, max_depth):
    root = get_split(train_dataset, attr_list, index_list)
    attr_list.append(root['idx'])
    node_split(root, 1,attr_list, max_depth)
    return root, attr_list

In [12]:
# to combine different folds of training set and get indexes of all rows
def get_foldset(updatedset, test_index): #, indexset):
    train_set = []
    indexes = []
    #print(updatedset)
    
    #remove the test set and combine other lists for train 
    newset = list(updatedset)
    newset.pop(test_index)
    train_set = sum(newset, [])
    
    #to randomly select rows equal to training set
    count = math.ceil(0.7 * len(train_set))
    rows = random.sample(range(0, len(train_set)-1), count)
    fold_set = []
    indices = []
    for row in rows:
        fold_set.append(train_set[row])
        indices.append(row)
    diff = len(train_set) - count
    rem_rows = random.sample(range(0, len(fold_set)-1), diff)
    left_set = []
    for row in rem_rows:
        fold_set.append(fold_set[row])
        indices.append(row)
    
    return fold_set, indices


In [13]:
# Prediction with a decision tree
def predict(node, row):
    if row[node['idx']] < node['a_val']:
        if isinstance(node['lft'], dict):
            return predict(node['lft'], row)
        else:
            return node['lft']
    else:
        if isinstance(node['rt'], dict):
            return predict(node['rt'], row)
        else:
            return node['rt']

In [14]:
def calc_results(majority_list, testdata):
    test_labels = []
    test_labels = [col[-1] for col in testdata]
    tp = 0 
    fn = 0 
    fp = 0 
    tn = 0
    for i in range(0, len(test_labels)):
        if test_labels[i] == 1:
            if majority_list[i] == 1: 
                tp += 1
            else:
                fn += 1
        else:
            if majority_list[i] == 1: 
                fp += 1
            else:
                tn += 1
    Accuracy = (tp + tn)/(tp + tn + fp + fn)
    if tp!= 0:
        Precision = tp/(tp + fp)
        Recall = tp/(tp + tn)
    else:
        Precision = 0
        Recall = 0

    F1_measure = (2*tp)/ (2*tp + tn + fp)
    
    return Accuracy, Precision, Recall, F1_measure

In [15]:
# check all outputs and return one with majority vote
def check_best(fold_root, testdata):
    majority = []
    for row in testdata:
        count0 = 0
        count1 = 0
        for root in fold_root:
            result_label = predict(root, row)
            if result_label == 1:
                count1 += 1
            else:
                count0 += 1
        if count0 > count1:
            majority.append(0)
        else:
            majority.append(1)
    results = calc_results(majority, testdata)
    return results
       

In [16]:
# main function
random.seed(1234)
tree_count = 5
n_folds = 10
max_depth = 5
updated_datasets = cross_validation(n_folds) #divide_dataset()
data_set = updated_datasets # [0]

train_set = []
count = 0
accuracies = []
precisions = []
recalls = []
f1_measures = []
for i in range(0, n_folds):
    fold_root = []
    count = 0
    print("beginning fold------------>" +str(i+1))
    
    while count < tree_count:
        #select one test data set from all the folds turn by turn
        test_set = data_set[i]
        fold_data = get_foldset(data_set, i) #, index_set)
        fold_treeset = fold_data[0]
        train_set = fold_data[0]
        index_list = fold_data[1]
        attribute_list = []
        # building tree using decision tree as base classifier
        tree_root = build_tree(fold_treeset, attribute_list, index_list, max_depth)
        fold_root.append(tree_root[0])
        count += 1
    best_tree = check_best(fold_root, test_set)
    accuracies.append(best_tree[0])
    precisions.append(best_tree[1])
    recalls.append(best_tree[2])
    f1_measures.append(best_tree[3])
    
    print("Fold accuracy::" + str(best_tree[0]))
    print("Fold precision::" + str(best_tree[1]))
    print("Fold recall::" + str(best_tree[2]))
    print("Fold f1 measure::" + str(best_tree[3]))

    
#after all the folds- find average of all the fold
resultant_accuracy = sum(accuracies)/ len(accuracies)
resultant_precision = sum(precisions)/ len(precisions)
resultant_recall = sum(recalls)/ len(recalls)
resultant_f1measure = sum(f1_measures)/len(f1_measures)

print("-----------------------------------------------------")
print("final accuracy::" + str(resultant_accuracy))
print("final precision::" + str(resultant_precision))
print("final recall::" + str(resultant_recall))
print("final f1::" + str(resultant_f1measure))
        

beginning fold------------>1
Fold accuracy::0.6086956521739131
Fold precision::0.6
Fold recall::0.21428571428571427
Fold f1 measure::0.3157894736842105
beginning fold------------>2
Fold accuracy::0.32608695652173914
Fold precision::0.32608695652173914
Fold recall::1.0
Fold f1 measure::0.4918032786885246
beginning fold------------>3
Fold accuracy::0.5652173913043478
Fold precision::0
Fold recall::0
Fold f1 measure::0.0
beginning fold------------>4
Fold accuracy::0.5
Fold precision::0.4074074074074074
Fold recall::0.4782608695652174
Fold f1 measure::0.44
beginning fold------------>5
Fold accuracy::0.3695652173913043
Fold precision::0.3695652173913043
Fold recall::1.0
Fold f1 measure::0.5396825396825397
beginning fold------------>6
Fold accuracy::0.6739130434782609
Fold precision::0
Fold recall::0
Fold f1 measure::0.0
beginning fold------------>7
Fold accuracy::0.6304347826086957
Fold precision::0
Fold recall::0
Fold f1 measure::0.0
beginning fold------------>8
Fold accuracy::0.7608695652